In [ ]:
import os
import sys
from pathlib import Path

# Add parent directory to Python path for imports
# This ensures we can import from _utils regardless of where the notebook is run from
current_dir = Path().resolve()
# If we're in the basics directory, go up one level; otherwise use current directory
if current_dir.name == 'basics':
    parent_dir = current_dir.parent
else:
    parent_dir = current_dir

if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

from openai import OpenAI
from _utils.website_scraper import fetch_website_contents
from _utils.standard_functions import load_env
from IPython.display import Markdown
from IPython.display import display, update_display


ModuleNotFoundError: No module named '_utils'

In [ ]:
# reading the env
load_env()

api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv("OPENAI_BASE_URL")

In [ ]:
# setting up the usual objects
openai = OpenAI(base_url=base_url, api_key=api_key)
model = "gpt-5-nano"
website_to_review="https://bramptonbjj.com"

To summarize any website, we need to break down our problem into the following sub-steps:

1. Scan all the available links in the website map - using a python library                <Standard Programing>
2. Ask AI to ignore the irrelevant links and retain information about the relevant pages   <AI Intelligence>
3. AI will scan the link pages and grab the relevant content                               <AI Intelligence>
4. AI will then summarize that info and return the details                                 <AI Intelligence>

Step 1 - List all the weblinks for the supplied website

In [ ]:
# custom function in the website_scrapper.py file
website_links = fetch_website_contents(website_to_review)
print(website_links)

Brazilian Jiu Jitsu - Brampton Braziian Jiu Jitsu Academy

HOME
BOOK A FREE CLASS
PRICE LIST
SCHEDULE
INSTRUCTORS
BRAZILIAN JIU JITSU
CONTACT
GEORGETOWN LOCATION
ORANGEVILLE LOCATION
More
HOME
BOOK A FREE CLASS
PRICE LIST
SCHEDULE
INSTRUCTORS
BRAZILIAN JIU JITSU
CONTACT
GEORGETOWN LOCATION
ORANGEVILLE LOCATION
HOME
BOOK A FREE CLASS
PRICE LIST
SCHEDULE
INSTRUCTORS
BRAZILIAN JIU JITSU
CONTACT
GEORGETOWN LOCATION
ORANGEVILLE LOCATION
Brampton Brazilian Jiu Jitsu Academy Inc.
Brampton Brazilian Jiu Jitsu Academy Inc.
Brampton Brazilian Jiu Jitsu Academy Inc.
Brampton Brazilian Jiu Jitsu Academy Inc.
Fostering positive lifestyles through Brazilian Jiu Jitsu
Brampton Brazilian Jiu Jitsu Academy Inc.
Brampton Brazilian Jiu Jitsu Academy Inc.
Brampton Brazilian Jiu Jitsu Academy Inc.
Brampton Brazilian Jiu Jitsu Academy Inc.
Fostering positive lifestyles through Brazilian Jiu Jitsu
Free Trial
Our Schedule
Our Classes
It is important to us that Brampton BJJ is the right fit for you before you 

We're going to ask the AI to fetch the details, by passing on instructions in the user prompt

In [18]:
user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.
"""

In [19]:
system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

Like before, we'll create the messages object and call the OpenAI object

In [20]:
def messages(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [ ]:
messages(website_to_review)

[{'role': 'system',
  'content': '\nYou are a snarky assistant that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\nhttps://bramptonbjj.com'}]

We will use the similar implementation to call OpenAPI as before, but we'll just wrap it in a function to make it easier for us

In [33]:
def summarize_website(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(model = model, messages = messages(website))
    return response.choices[0].message.content

In [25]:
summarize_website(website_to_review)

'# Brampton Brazilian Jiu Jitsu Academy: Where Chokes Meet Chuckles\n\nWelcome to the Brampton Brazilian Jiu Jitsu Academy, your one-stop shop for knocking people (or tackling life) to the mat with style. They’re all about "fostering positive lifestyles"—because nothing says positivity like a good armbar.\n\n**Highlights:**\n- **Free Trial**: Dip your toes into the Jiu Jitsu pool with a free class, a semi-private lesson, and a complimentary loaner uniform. Perfect for pros and absolute newbies who just want to avoid embarrassment on their first day.\n- **Classes for Everyone**: Kids, adults, men, women, couch potatoes, and future MMA champs alike.\n- **Flexible Schedule**: Early birds, lunch-break warriors, and night ninjas welcome.\n- **Locations**: Brampton main spot with branches in Georgetown and Orangeville, because who doesn’t want options when it’s time to wrestle strangers?\n\nNo recent news or earth-shattering announcements, just the regular vibe: come train, get fit, learn co

In [39]:
# Make it presentable
def summarize_website_cleanly(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(model = model, messages = messages(website))
    return display(Markdown(response.choices[0].message.content))

In [35]:
summarize_website_cleanly(website_to_review)

- Short version: It’s Brampton Brazilian Jiu Jitsu Academy’s site, basically a brochure for a gym that wants to turn you into a grappling ninja. It hawks a free trial, a loaner gi, and a schedule that supposedly fits any chaotic life, with multiple locations and a pile of pages to click through.

What it offers (without the navigation noise):
- Free trial: includes a semi-private intro and a free loaner uniform.
- Programs: classes for kids, adults, all genders, all experience levels, including pro competitors.
- Schedule: early mornings, noon slots, and evenings to accommodate busy schedules.
- Locations: Brampton main location plus Georgetown and Orangeville.
- Extras: price list, instructors, contact info, and a signup form (with a recaptcha/privacy policy caveat).

News / announcements:
- The site shows an old-looking timestamp (14-12-19) near the trial offer, which suggests the big “book your free trial” push is not exactly fresh. Other than that, no obvious current news or updates are highlighted.

Snarky takeaway: If you’re into grappling with life and a membership that promises “fostering positive lifestyles,” this site will guide you to a free trial faster than you can say “kimura.”

In [ ]:
# Make a fancy summary
def summarize_website_fancy(url):
    website = fetch_website_contents(url)
    stream = openai.chat.completions.create(model = model, messages = messages(website), stream=True)

    response = ""

    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [38]:
summarize_website_fancy(website_to_review)

Brampton Brazilian Jiu Jitsu Academy is a multi-location gym offering beginner-friendly BJJ for kids, adults, and pros, with a big emphasis on trying it first. Their angle: free trials with a semi-private intro lesson and a loaner uniform, plenty of class options, and schedules that pretend to fit busy lives. They push “fostering positive lifestyles” through Brazilian Jiu Jitsu, and you can reach them at the Brampton address (plus Georgetown and Orangeville spots), via email, or by phone. News-wise, there isn’t a flashy announcements section—just the recurring call-to-action to book a free trial. 

- Free trial: semi-private intro lesson + loaner gi
- Classes for all levels and ages (kids, adults, beginners, competitors)
- Flexible schedule: early morning, noon, and evening
- Locations: Brampton (main), plus Georgetown and Orangeville
- Contact: info@bramptonbjj.ca, 416-230-9653
- Extras: price list, instructors, schedule, and a note about recaptcha/privacy terms

If you’re lurker-curious about BJJ, this site basically says “try it for free, then decide.”